In [1]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.constants as const
import torch
from tqdm import tqdm
import pandas as pd

c:\Users\zhantao\.conda\envs\quspin\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_lst = glob.glob('../../SpinW_data_generation/MATLAB_SpinW/CrI3_data/20221110/*.mat')
print(f"Find {len(file_lst)} files.")

Find 1000 files.


In [3]:
q_idx = 0
param_lst = []
omega_lst = []
inten_lst = []
for file in tqdm(file_lst):
    data = scipy.io.loadmat(file)
    param = torch.tensor([data["J1"][0,0], data["DM"][0,0], data["K"][0,0]])
    nz_idx = np.nonzero(data['swConv'][:,q_idx])[0]
    omega = torch.from_numpy(data['Evect'][0, nz_idx])
    inten = torch.from_numpy(data['swConv'][nz_idx, q_idx])

    if len(omega) == 1:
        omega = torch.cat((omega, omega), dim=0)
        inten = torch.cat((inten/2, inten/2), dim=0)
    
    if len(omega) <= 2:
        param_lst.append(param)
        omega_lst.append(omega)
        inten_lst.append(inten)


100%|██████████| 1000/1000 [00:05<00:00, 169.34it/s]


In [4]:
save_dict = {
    "param": torch.vstack(param_lst),
    "omega": torch.vstack(omega_lst),
    "inten": torch.vstack(inten_lst),
    "note": "J1=[-2.5,0]; DM=[-1.0,0]; K=[0 0]; Jc=-0.6; Dz=-0.1; All zeros otherwise;"
}

In [5]:
torch.save(save_dict, "data/CrI3/20221110.pt")